In [1]:
import add_path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

In [2]:
from transformers import AutoTokenizer
from models.BERT.tucoregcn_bert_pytorch import TUCOREGCN_BertForSequenceClassification, TUCOREGCN_BertConfig
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tucore_gcn_bert = TUCOREGCN_BertForSequenceClassification(TUCOREGCN_BertConfig.from_json_file("../models/BERT/tucoregcn_bert_mlc.json"))
tucore_gcn_bert.cuda()

d:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


d:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\dgl\dgl.dll


TUCOREGCN_BertForSequenceClassification(
  (tucoregcn_bert): TUCOREGCN_Bert(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (speaker_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutpu

In [3]:
tucore_gcn_bert.load_state_dict(torch.load("../TUCOREGCN_BERT_DialogRE/tucoregcn_pytorch_model.pt", map_location="cuda"))
tucore_gcn_bert.cuda()

TUCOREGCN_BertForSequenceClassification(
  (tucoregcn_bert): TUCOREGCN_Bert(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (speaker_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutpu

In [4]:
from data import TUCOREGCNDataset, TUCOREGCNDataloader
dev_set = TUCOREGCNDataset(src_file="../datasets/DialogRE/dev.json", save_file="../datasets/DialogRE/dev_BERT.pkl", max_seq_length=512, tokenizer=tokenizer, n_class=36, encoder_type='BERT')
dev_loader = TUCOREGCNDataloader(dataset=dev_set, batch_size=2, shuffle=False, relation_num=36, max_length=512)
idx, batch = next(enumerate(dev_loader))
entry_idx = 0
input_ids = batch["input_ids"].to("cuda:0")
segment_ids = batch["segment_ids"].to("cuda:0")
input_mask = batch["input_masks"].to("cuda:0")
speaker_ids = batch["speaker_ids"].to("cuda:0")
graphs = batch["graphs"]
mention_id = batch["mention_ids"].to("cuda:0")
turn_mask = batch["turn_masks"].to("cuda:0")
output = tucore_gcn_bert(
	input_ids,
	segment_ids,
	input_mask,
	speaker_ids,
	graphs,
	mention_id,
	None,
	turn_mask
)
output

Reading data from ../datasets/DialogRE/dev.json.
load preprocessed data from ../datasets/DialogRE/dev_BERT.pkl.


{'graph_output': tensor([[ 0.4662, -0.0104,  0.6122,  ...,  0.0000,  0.0000,  0.0288],
         [ 0.2737,  0.0785,  0.2930,  ...,  2.1276,  0.4612,  3.3512]],
        device='cuda:0', grad_fn=<StackBackward0>),
 'graphs': [Graph(num_nodes={'node': 23},
        num_edges={('node', 'dialog', 'node'): 40, ('node', 'entity', 'node'): 24, ('node', 'speaker', 'node'): 114},
        metagraph=[('node', 'node', 'dialog'), ('node', 'node', 'entity'), ('node', 'node', 'speaker')]),
  Graph(num_nodes={'node': 23},
        num_edges={('node', 'dialog', 'node'): 40, ('node', 'entity', 'node'): 10, ('node', 'speaker', 'node'): 114},
        metagraph=[('node', 'node', 'dialog'), ('node', 'node', 'entity'), ('node', 'node', 'speaker')])],
 'sequence_outputs': tensor([[[ 0.1739, -0.7403, -0.0519,  ...,  0.7922, -1.2172, -0.3233],
          [ 0.0245, -0.1028,  0.4226,  ...,  0.7190, -0.4245, -0.9714],
          [-0.4540, -0.1118,  0.0846,  ...,  1.1896, -1.2328, -1.0474],
          ...,
          [ 0.0

In [5]:
import dgl
sequence_output = output["sequence_outputs"][0]
pooled_output = output["pooled_outputs"][0]
mention_id = batch["mention_ids"].to("cuda:0")[0]

#initialize some variables
slen = input_ids.size(1)
# Find the last turn (in the case of tucoregcn, it is the masked speaker dictionary)
mention_num = torch.max(mention_id)
# Create a mention matrix idx of mention_num*slen
num_batch_turn = mention_num + 1
mention_index = (
	(torch.arange(mention_num) + 1).unsqueeze(1).expand(-1, slen)
)
if torch.cuda.is_available():
	mention_index = mention_index.cuda()
# Create a mentions matrix of slen*mention_num
mentions = mention_id.unsqueeze(0).expand(mention_num, -1)
# Generate 1 hot encoding of each speaker's dialogue
select_metrix = (mention_index == mentions).float()
# Factor into the one-hot encoding, the total number of words in each speaker's dialogue
word_total_numbers = (
	torch.sum(select_metrix, dim=-1).unsqueeze(-1).expand(-1, slen)
)
select_metrix = torch.where(
	word_total_numbers > 0,
	select_metrix / word_total_numbers,
	select_metrix,
)
# Apply one hot encoding to sequence_output to selectively obtain features
features = torch.mm(select_metrix, sequence_output)
features = torch.cat((pooled_output.unsqueeze(0), features), dim=0)

In [94]:
val, ind = torch.max(torch.LongTensor([[1,2,3],[1,2,4]]),dim=1)
ind

tensor([2, 2])

In [29]:
torch.ones(3,5)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [30]:
mention_nums

tensor([22, 22], device='cuda:0')

tensor([[6, 6, 6, 6, 6, 6, 6, 6],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [8, 8, 8, 8, 8, 8, 8, 8]])

In [51]:
temp = torch.LongTensor([6,7,8])
result = torch.arange(0, torch.max(temp)+1).expand(len(temp), -1)
result>temp.expand(torch.max(temp)+1,-1).T

tensor([[False, False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False, False,  True],
        [False, False, False, False, False, False, False, False, False]])

In [16]:
sequence_outputs = output["sequence_outputs"]
pooled_outputs = output["pooled_outputs"]
graphs = batch["graphs"]
mention_ids = batch["mention_ids"].to("cuda:0")
mention_ids
#initialize some variables
slen = input_ids.size(1)
# Find the last turn (in the case of tucoregcn, it is the masked speaker dictionary)
mention_nums, _ = torch.max(mention_ids,dim=1)
num_batch_turns = mention_nums + 1
'''
mention_index = (
	(torch.arange(mention_nums,dim=1) + 1).unsqueeze(1).expand(-1, slen)
)
if torch.cuda.is_available():
	mention_index = mention_index.cuda()'''
mentions = mention_ids.unsqueeze(1).expand(-1, mention_num, -1)
mentions

TypeError: arange() received an invalid combination of arguments - got (Tensor, dim=int), but expected one of:
 * (Number end, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (Number start, Number end, *, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (Number start, Number end, Number step, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [65]:
mention_id.unsqueeze(0).expand(mention_num, -1)

tensor([[ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        ...,
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0]], device='cuda:0')

In [71]:
mention_ids.unsqueeze(1).expand(-1, mention_num, -1)[0]

tensor([[ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        ...,
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0]], device='cuda:0')

In [64]:
input_ids.size(1)

512

In [78]:
torch.sum((mention_index==mentions).float(), dim=-1).unsqueeze(-1).expand(-1, slen)

tensor([[ 4.,  4.,  4.,  ...,  4.,  4.,  4.],
        [ 5.,  5.,  5.,  ...,  5.,  5.,  5.],
        [12., 12., 12.,  ..., 12., 12., 12.],
        ...,
        [75., 75., 75.,  ..., 75., 75., 75.],
        [ 2.,  2.,  2.,  ...,  2.,  2.,  2.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]], device='cuda:0')

In [80]:
select_metrix[1]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2000, 0.2000, 0.2000, 0.2000,
        0.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [76]:
(mention_index==mentions).float()[1]

tensor([0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [70]:
select_metrix

tensor([[0.0000, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000]],
       device='cuda:0')

In [68]:
mentions.shape

torch.Size([22, 512])

In [69]:
(torch.arange(mention_num) + 1).unsqueeze(1).expand(-1, slen).shape

torch.Size([22, 512])

In [58]:
tokenizer.convert_ids_to_tokens(input_ids[0])

['[CLS]',
 '[unused2]',
 ':',
 'hey',
 '!',
 'speaker',
 '2',
 ':',
 'hey',
 '.',
 'speaker',
 '3',
 ':',
 'hey',
 ',',
 'man',
 '.',
 'what',
 "'",
 's',
 'up',
 '?',
 '[unused2]',
 ':',
 'maybe',
 'you',
 'can',
 'tell',
 'me',
 '.',
 'my',
 'agent',
 'would',
 'like',
 'to',
 'know',
 'why',
 'i',
 'didn',
 "'",
 't',
 'show',
 'up',
 'at',
 'the',
 'audition',
 'i',
 'didn',
 "'",
 't',
 'know',
 'i',
 'had',
 'today',
 '.',
 'the',
 'first',
 'good',
 'thing',
 'she',
 'gets',
 'me',
 'in',
 'weeks',
 '.',
 'how',
 'could',
 'you',
 'not',
 'give',
 'me',
 'the',
 'message',
 '?',
 '!',
 'speaker',
 '3',
 ':',
 'well',
 ',',
 'i',
 "'",
 'll',
 'tell',
 'ya',
 'i',
 'do',
 'enjoy',
 'guilt',
 ',',
 'but',
 ',',
 'ah',
 ',',
 'it',
 'wasn',
 "'",
 't',
 'me',
 '.',
 'speaker',
 '2',
 ':',
 'yes',
 ',',
 'it',
 'was',
 '!',
 'it',
 'was',
 'him',
 '!',
 'uh',
 'huh',
 '!',
 'okay',
 ',',
 'it',
 'was',
 'me',
 '!',
 '[unused2]',
 ':',
 'how',
 'is',
 'it',
 'you',
 '?',
 'speaker',


In [57]:
turn_mask[0]

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1]], device='cuda:0')

In [52]:
mention_id

tensor([ 0,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,
         7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 

In [43]:
mentions.shape

torch.Size([22, 512])

In [38]:
mention_id.shape

torch.Size([512])

In [39]:
mention_index.shape

torch.Size([22, 512])

In [7]:
state_dict = torch.load("../TUCOREGCN_BERT_DialogRE/tucoregcn_pytorch_model.pt", map_location="cuda")
for key in list(state_dict.keys()):
    state_dict[key.replace("tucoregcn_bert.classifier.", "classifier.")] = state_dict.pop(key)

In [8]:
torch.save(state_dict, "../TUCOREGCN_BERT_DialogRE/tucoregcn_pytorch_model.pt")